<a href="https://colab.research.google.com/github/davidlealo/TalkTrack/blob/main/talk_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Instalación de libraries en Colab

In [ ]:
!pip install opencv-python-headless numpy openai-whisper moviepy torch torchvision
!apt-get install -y ffmpeg


Carga de a las libraries a utilizar

In [2]:
import cv2
import whisper
from google.colab import files
from moviepy.editor import VideoFileClip
from datetime import datetime
import numpy as np
import os

  if event.key is 'enter':



In [ ]:


# Configuración inicial
VIDEO_FILE = "uploaded_video.mov"
AUDIO_FILE = "extracted_audio.wav"
TRANSCRIPTION_FILE = "transcription_with_speakers.txt"
TEMP_VIDEO_FRAMES = "temp_frames"

# Cargar el modelo Whisper
whisper_model = whisper.load_model("base")

def upload_video():
    """Permite subir un archivo de video en formato .mov"""
    print("Por favor, sube un archivo de video en formato .mov.")
    uploaded = files.upload()
    if VIDEO_FILE in uploaded:
        print(f"Video {VIDEO_FILE} cargado exitosamente.")
    else:
        raise FileNotFoundError("No se encontró el archivo de video esperado.")

def extract_audio_from_video(video_path, audio_path):
    """Extrae el audio de un archivo de video y lo guarda como .wav"""
    try:
        print("Extrayendo audio del video...")
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_path)
        print(f"Audio extraído y guardado en: {audio_path}")
    except Exception as e:
        raise RuntimeError(f"Error al extraer el audio: {e}")

def process_video_frames(video_path):
    """Procesa los fotogramas del video para detectar personas y cambios de hablantes."""
    if not os.path.exists(TEMP_VIDEO_FRAMES):
        os.makedirs(TEMP_VIDEO_FRAMES)

    print("Procesando fotogramas del video...")
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = 0

    speaker_changes = []
    previous_frame = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % fps == 0:  # Procesar 1 frame por segundo
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if previous_frame is not None:
                diff = cv2.absdiff(previous_frame, gray_frame)
                non_zero_count = np.count_nonzero(diff)

                if non_zero_count > 10000:  # Umbral para detectar cambios significativos
                    speaker_changes.append({
                        "time": frame_count / fps,
                        "description": "Cambio de hablante detectado"
                    })

            previous_frame = gray_frame

        frame_count += 1

    cap.release()
    print("Procesamiento de fotogramas completado.")
    return speaker_changes

def transcribe_audio_with_speakers(audio_path, speaker_changes):
    """Transcribe el audio y etiqueta segmentos con cambios de hablantes."""
    print("Iniciando la transcripción...")
    try:
        result = whisper_model.transcribe(audio_path, language="es")
        segments = result["segments"]
        transcription = []

        for segment in segments:
            start_time = segment["start"]
            end_time = segment["end"]
            text = segment["text"]

            # Determinar el hablante en función de los cambios detectados
            speaker = "Desconocido"
            for change in speaker_changes:
                if change["time"] <= start_time <= (change["time"] + 1):
                    speaker = change["description"]
                    break

            transcription.append(f"[{start_time:.2f}-{end_time:.2f}] ({speaker}): {text}")

        with open(TRANSCRIPTION_FILE, "w") as f:
            f.write("\n".join(transcription))
        print(f"Transcripción completada y guardada en {TRANSCRIPTION_FILE}")

    except Exception as e:
        raise RuntimeError(f"Error durante la transcripción: {e}")

def main():
    try:
        upload_video()
        extract_audio_from_video(VIDEO_FILE, AUDIO_FILE)
        speaker_changes = process_video_frames(VIDEO_FILE)
        transcribe_audio_with_speakers(AUDIO_FILE, speaker_changes)
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()
